https://linqs.soe.ucsc.edu/data

In [1]:
import torch
import numpy as np
import scipy.sparse as sp
import networkx as nx
import pickle as pkl
import random as rd
import pandas as pd
import warnings; warnings.filterwarnings('ignore')

### Import raw data

In [3]:
with open('data/raw/pubmed.content.tab') as f:
    reader = csv.reader(f, delimiter="\t")
    feat = list(reader)

#### build dataframe

In [4]:
cols = ['id']
cols = cols + feat[1][1:-1]
cols.append('label')

In [97]:
len(feat[2:])

19717

In [133]:
feat_df = pd.DataFrame('0', index=np.arange(len(feat[2:])), columns=cols)

In [134]:
feat_df.columns = feat_df.columns.str.replace('numeric:', '')
feat_df.columns = feat_df.columns.str.replace(':0.0', ''); feat_df

,id,w-rat,w-common,w-use,w-examin,w-pathogenesi,w-retinopathi,w-mous,w-studi,w-anim,...,w-urinari,w-myocardi,w-meal,w-ica,w-locus,w-tcell,w-depress,w-bone,w-mutat,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19712,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19715,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [135]:
for i in range(len(all_feat)):
    id_ = all_feat[i][0] 
    label = all_feat[i][1].split('=')[1]
    values = all_feat[i][2:-1]
    
    feat_df.loc[i]['id'] = id_
    feat_df.loc[i]['label'] = label
    for j in values:
        word = j.split('=')[0]
        tfidf_value = j.split('=')[1]
        feat_df.loc[i][word] = feat_df.loc[i][word].replace('0', tfidf_value)

In [142]:
feat_df['id']  = feat_df['id'].astype(str)
feat_df['label']  = feat_df['label'].astype(str)
feat_df.iloc[:,1:-1]  = feat_df.iloc[:,1:-1].astype(float)

In [143]:
feat_df

,id,w-rat,w-common,w-use,w-examin,w-pathogenesi,w-retinopathi,w-mous,w-studi,w-anim,...,w-urinari,w-myocardi,w-meal,w-ica,w-locus,w-tcell,w-depress,w-bone,w-mutat,label
0,12187484,0.093935,0.028698,0.01176,0.019375,0.063161,0.170891,0.067702,0.017555,0.098402,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,2344352,0.023618,0.0,0.014784,0.0,0.0,0.0,0.0,0.0,0.030926,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,14654069,0.102263,0.0,0.010669,0.0,0.0,0.0,0.0,0.0,0.044636,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,16443886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2684155,0.030616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.080179,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19712,17559889,0.0,0.0,0.022998,0.0,0.0,0.0,0.0,0.022886,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
19713,8792097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008728,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
19714,17934141,0.067942,0.0,0.014177,0.0,0.0,0.0,0.0,0.0,0.059311,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
19715,18673544,0.0,0.0,0.00924,0.0,0.0,0.0,0.0,0.009195,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [180]:
feature = feat_df.iloc[:,1:-1]
feature = np.array(feature, dtype=float)
sp_feature = sp.csr_matrix(feature)

In [183]:
# with open('data/pubmed.feature', 'wb') as f:
#     pkl.dump(sp_feature, f)

### Adjacency matrix

In [147]:
import csv
with open('data/raw/pubmed.cites.tab') as f:
    reader = csv.reader(f, delimiter="\t")
    edge = list(reader)[2:]

In [156]:
elist = pd.DataFrame(edge, columns=['N1', 'source','N2', 'target'])
elist = elist[['source', 'target']]; 
elist['source'] = elist['source'].str.replace('paper:', '')
elist['target'] = elist['target'].str.replace('paper:', ''); elist

,source,target
0,19127292,17363749
1,19668377,17293876
2,1313726,3002783
3,19110882,14578298
4,18606979,10333910
...,...,...
44333,18776148,1473616
44334,2040701,3894118
44335,19036853,10864585
44336,18078023,15270789


In [157]:
G = nx.from_pandas_edgelist(elist, 'source', 'target')
connected_nodes_list = list(G.nodes())

In [163]:
feat_df['id'][3]

'16443886'

In [164]:
feat_idx2drop = []
for i in feat_df.index:
    if feat_df['id'][i] not in connected_nodes_list:
        feat_idx2drop.append(i)

In [166]:
name2id = {}
for i in feat_df.index:
    name2id[feat_df['id'][i]] = i

In [167]:
elist_id = elist.copy()
idx2drop = []
for i in elist_id.index:
    try:
        elist_id.iloc[i][0] = name2id[elist_id.iloc[i][0]]
        elist_id.iloc[i][1] = name2id[elist_id.iloc[i][1]]
    except KeyError:
        idx2drop.append(i)

In [168]:
len(idx2drop)

0

In [169]:
elist_id = elist_id.drop(elist_id.index[idx2drop])
elist_id = elist_id.reset_index(drop=True); elist_id

,source,target
0,10816,18459
1,8069,3834
2,297,16724
3,8613,10896
4,9238,12701
...,...,...
44333,8928,15416
44334,2504,12742
44335,9535,13533
44336,19250,5209


In [170]:
G = nx.from_pandas_edgelist(elist_id, 'source', 'target')

In [171]:
len(G.nodes())

19717

In [172]:
adj = nx.adjacency_matrix(G, nodelist=sorted(G.nodes()))
adj = adj.todense()

In [173]:
np.fill_diagonal(adj, 0)
adj = sp.csr_matrix(adj)

In [174]:
# with open('data/pubmed.graph', 'wb') as f:
#     pkl.dump(adj, f)

### Labels

In [184]:
labels = feat_df.iloc[:,-1] 
labels = labels.values; labels

array(['1', '1', '1', ..., '1', '3', '3'], dtype=object)

In [185]:
# with open('data/pubmed.labels', 'wb') as f:
#     pkl.dump(labels, f)